In [1]:
import pandas as pd

In [3]:
messages = pd.read_csv('SMSSpamCollection',sep='\t',names=['label','message'])

In [5]:
messages.head(10)

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...
6,ham,Even my brother is not like to speak with me. ...
7,ham,As per your request 'Melle Melle (Oru Minnamin...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...


In [10]:
messages.query('label=="spam"').count()

label      747
message    747
dtype: int64

In [12]:
messages.query('label=="ham"').count()

label      4825
message    4825
dtype: int64

In [17]:
messages['message'].loc[100]

"Please don't text me anymore. I have nothing else to say."

In [18]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [23]:
stemmer = PorterStemmer()

In [26]:
corpus = []
for msg in messages['message']:
    review = re.sub('[^a-zA-Z]',' ',msg).lower()
    review = review.split()

    review = [stemmer.stem(word) for word in review if word not in stopwords.words('english')]
    corpus.append(" ".join(review))

In [29]:
corpus[:10]

['go jurong point crazi avail bugi n great world la e buffet cine got amor wat',
 'ok lar joke wif u oni',
 'free entri wkli comp win fa cup final tkt st may text fa receiv entri question std txt rate c appli',
 'u dun say earli hor u c alreadi say',
 'nah think goe usf live around though',
 'freemsg hey darl week word back like fun still tb ok xxx std chg send rcv',
 'even brother like speak treat like aid patent',
 'per request mell mell oru minnaminungint nurungu vettam set callertun caller press copi friend callertun',
 'winner valu network custom select receivea prize reward claim call claim code kl valid hour',
 'mobil month u r entitl updat latest colour mobil camera free call mobil updat co free']

In [32]:
messages['message'][0]

'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'

# BOW

In [34]:
from sklearn.feature_extraction.text import CountVectorizer

In [71]:
cv = CountVectorizer(max_features=2500,ngram_range=(1,2),binary=True)

In [72]:
X = cv.fit_transform(corpus).toarray()

In [73]:
X[0]

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [74]:
y = pd.get_dummies(messages['label'])

In [75]:
y = y.iloc[:,1].values

In [76]:
y

array([0, 0, 1, ..., 0, 0, 0], dtype=uint8)

In [77]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report

In [78]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [79]:
model = MultinomialNB()

In [80]:
model.fit(X_train,y_train)

MultinomialNB()

In [81]:
preds = model.predict(X_test)

In [82]:
accuracy_score(y_test,preds)

0.9865470852017937

In [83]:
print(classification_report(preds,y_test))

              precision    recall  f1-score   support

           0       1.00      0.99      0.99       973
           1       0.93      0.97      0.95       142

    accuracy                           0.99      1115
   macro avg       0.96      0.98      0.97      1115
weighted avg       0.99      0.99      0.99      1115



# TF-IDF

In [99]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [100]:
tf = TfidfVectorizer(max_features=2500,ngram_range=(1,2))
X = tf.fit_transform(corpus).toarray()

In [101]:
X.shape

(5572, 2500)

In [102]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [103]:
model.fit(X_train,y_train)
preds = model.predict(X_test)

In [104]:
print(accuracy_score(preds,y_test))
print(classification_report(preds,y_test))

0.9820627802690582
              precision    recall  f1-score   support

           0       1.00      0.98      0.99       984
           1       0.87      0.99      0.93       131

    accuracy                           0.98      1115
   macro avg       0.94      0.99      0.96      1115
weighted avg       0.98      0.98      0.98      1115



# Word2Vec

In [105]:
from nltk.stem import WordNetLemmatizer
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

In [107]:
lemmatizer = WordNetLemmatizer()

In [116]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\tejas\AppData\Roaming\nltk_data...


True

In [167]:
corpus = []
for msg in messages['message']:
    review = re.sub('[^a-zA-Z]',' ',msg).lower()
    review = review.split()

    review = [lemmatizer.lemmatize(word) for word in review if word not in stopwords.words('english')]
    corpus.append(" ".join(review))

In [193]:
corpus[:10]

['go jurong point crazy available bugis n great world la e buffet cine got amore wat',
 'ok lar joking wif u oni',
 'free entry wkly comp win fa cup final tkts st may text fa receive entry question std txt rate c apply',
 'u dun say early hor u c already say',
 'nah think go usf life around though',
 'freemsg hey darling week word back like fun still tb ok xxx std chgs send rcv',
 'even brother like speak treat like aid patent',
 'per request melle melle oru minnaminunginte nurungu vettam set callertune caller press copy friend callertune',
 'winner valued network customer selected receivea prize reward claim call claim code kl valid hour',
 'mobile month u r entitled update latest colour mobile camera free call mobile update co free']

In [169]:
words = [simple_preprocess(msg) for msg in corpus]

In [171]:
words[0]

['go',
 'jurong',
 'point',
 'crazy',
 'available',
 'bugis',
 'great',
 'world',
 'la',
 'buffet',
 'cine',
 'got',
 'amore',
 'wat']

In [172]:
import gensim

In [173]:
model = gensim.models.Word2Vec(words,min_count=2,window=5)

In [194]:
# model.wv.index_to_key

In [175]:
model.corpus_count

5572

In [176]:
type(model)

gensim.models.word2vec.Word2Vec

In [177]:
model.wv.most_similar('sorry')

[('call', 0.9993900656700134),
 ('msg', 0.9992949366569519),
 ('please', 0.999291718006134),
 ('see', 0.9992879629135132),
 ('back', 0.9992780089378357),
 ('get', 0.9992556571960449),
 ('babe', 0.9992413520812988),
 ('amp', 0.9992395639419556),
 ('one', 0.9992306232452393),
 ('need', 0.9992299675941467)]

In [178]:
model.wv['back'].shape

(100,)

In [179]:
import numpy as np

In [180]:
def avg_word2vec(doc):
    vectors = [model.wv[word] for word in doc if word in model.wv.index_to_key]
    if len(vectors) == 0:
        return np.zeros(model.vector_size)
    return np.mean(vectors, axis=0)

In [181]:
words[102]

['still', 'looking', 'car', 'buy', 'gone', 'driving', 'test', 'yet']

In [182]:
X = []
for i in words:
    X.append(avg_word2vec(i))

In [183]:
X_new = np.array(X)

In [184]:
y = messages['label'].map({'ham': 0, 'spam': 1}).values

In [185]:
X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.2, random_state=42)

In [189]:
from sklearn.ensemble import RandomForestClassifier

In [190]:
model = RandomForestClassifier()

In [191]:
model.fit(X_train,y_train)
preds = model.predict(X_test)

In [192]:
print(accuracy_score(preds,y_test))
print(classification_report(preds,y_test))

0.9641255605381166
              precision    recall  f1-score   support

           0       0.99      0.97      0.98       994
           1       0.77      0.95      0.85       121

    accuracy                           0.96      1115
   macro avg       0.88      0.96      0.92      1115
weighted avg       0.97      0.96      0.97      1115

